<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=6cdfb9996c7f2cc8be1821635107d97bdbb95bdd478dd3db3b1780f600b283d7
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-700)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['ACC', 'ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANITRANS',
       'AMBUJACEM', 'APOLLOHOSP', 'ASIANPAINT', 'AUBANK', 'AUROPHARMA',
       'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE',
       'BANDHANBNK', 'BANKBARODA', 'BATAINDIA', 'BERGEPAINT',
       'BHARTIARTL', 'BIOCON', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'CANBK',
       'CHOLAFIN', 'CIPLA', 'COALINDIA', 'COLPAL', 'DABUR', 'DEEPAKNTR',
       'DIVISLAB', 'DLF', 'DMART', 'DRREDDY', 'EICHERMOT', 'EMAMILTD',
       'FEDERALBNK', 'GAIL', 'GILLETTE', 'GLAND', 'GODREJCP', 'GODREJIND',
       'GRASIM', 'HAVELLS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK',
       'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDPETRO', 'HINDUNILVR',
       'IBREALEST', 'ICICIBANK', 'ICICIGI', 'ICICIPRULI', 'IGL',
       'INDIANB', 'INDIGO', 'INDIGOPNTS', 'INDUSINDBK', 'INDUSTOWER',
       'INFY', 'IOC', 'IRCTC', 'ITC', 'JINDALSTEL', 'JSWSTEEL',
       'JUBLFOOD', 'KOTAKBANK', 'LT', 'LTIM', 'LUPIN', 'M&M', 'MARICO',
       'MARUTI', '

In [4]:
new_stars = ['ADANIENT.NS', 'LTIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
              'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IGL.NS', 'INDIANB.NS',
              'INDIGOPNTS.NS', 'ADANIGREEN.NS', 'INFY.NS', 'IRCTC.NS', 'ITC.NS',
              'JUBLFOOD.NS', 'M&M.NS', 'GODREJIND.NS', 'MARICO.NS', 'MCDOWELL-N.NS',
              'NTPC.NS', 'PAGEIND.NS', 'PIDILITIND.NS', 'PIIND.NS', 'RELIANCE.NS',
              'SBICARD.NS', 'SBIN.NS', 'TATACONSUM.NS', 'TATAPOWER.NS', 'TCS.NS',
              'TITAN.NS', 'UPL.NS', 'GRASIM.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS',
              'CANBK.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'BERGEPAINT.NS', 'COALINDIA.NS',
              'AUBANK.NS', 'BAJAJ-AUTO.NS', 'DMART.NS', 'BRITANNIA.NS', 'DRREDDY.NS',
              'ASIANPAINT.NS', 'EICHERMOT.NS', 'BAJFINANCE.NS', 'BANDHANBNK.NS',
              'FEDERALBNK.NS', 'BHARTIARTL.NS']

curr_stars = ['ADANIENT.NS', 'LTIM.NS', 'HCLTECH.NS', 'HDFC.NS', 'HDFCBANK.NS',
               'HINDUNILVR.NS', 'ICICIBANK.NS', 'ICICIGI.NS', 'IGL.NS', 'INDIANB.NS',
               'INDIGOPNTS.NS', 'ADANIGREEN.NS', 'INFY.NS', 'IRCTC.NS', 'ITC.NS',
               'JUBLFOOD.NS', 'M&M.NS', 'GODREJIND.NS', 'MARICO.NS', 'MCDOWELL-N.NS',
               'NTPC.NS', 'PAGEIND.NS', 'PIDILITIND.NS', 'PIIND.NS', 'RELIANCE.NS',
               'SBICARD.NS', 'SBIN.NS', 'TATACONSUM.NS', 'TATAPOWER.NS', 'TCS.NS',
               'TITAN.NS', 'UPL.NS', 'GRASIM.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS',
               'CANBK.NS', 'CHOLAFIN.NS', 'CIPLA.NS', 'BERGEPAINT.NS', 'COALINDIA.NS',
               'AUBANK.NS', 'BAJAJ-AUTO.NS', 'DMART.NS', 'BRITANNIA.NS', 'DRREDDY.NS',
               'ASIANPAINT.NS', 'EICHERMOT.NS', 'BAJFINANCE.NS', 'BANDHANBNK.NS',
               'FEDERALBNK.NS', 'BHARTIARTL.NS']

print('entries:', list(set(new_stars) - set(curr_stars)))
print('exits:', list(set(curr_stars) - set(new_stars)))

entries: []
exits: []


In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

L    67
H    31
M    27
Name: Conviction, dtype: int64

In [6]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    110
1     15
Name: InPortfolio, dtype: int64

In [7]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"


  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [9]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 150
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_150'] = round((stock_df['Close'] - stock_df['150_SMA'])*100/stock_df['150_SMA'],2)
    stock_df['150_SMA_Lag10'] = stock_df['150_SMA'].shift(10)
    stock_df['Slope%_150'] = round((stock_df['150_SMA'] - stock_df['150_SMA_Lag10'])*100/stock_df['150_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    stock_df.ta.rsi(append=True)
    stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','150_SMA','150_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    #tmp['Curr_PE'] = current_pe
    tmp['EstmtFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['EstmtFlag'].values[0] 
    tmp['DuraFlag'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['DuraFlag'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]      
    df_prec_dev = df_prec_dev.append(tmp)

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,,
2023-03-27 00:00:00+05:30,16986.0,^NSEI,-4.61,-0.24,-0.49,37.0,18812.0,15294.0,51.90,11.0
2023-04-05 00:00:00+05:30,17557.0,^NSEI,-1.35,-0.20,-0.44,58.0,18812.0,15294.0,35.67,7.0
2023-04-13 00:00:00+05:30,17828.0,^NSEI,0.14,-0.02,-0.13,66.0,18812.0,15294.0,27.97,6.0
2023-04-21 00:00:00+05:30,17624.0,^NSEI,-0.98,0.01,-0.02,56.0,18812.0,15294.0,33.77,7.0
2023-04-28 00:00:00+05:30,18065.0,^NSEI,1.49,-0.02,0.02,71.0,18812.0,15294.0,21.23,4.0


In [11]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [12]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 15


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,EstmtFlag,DuraFlag,Conviction
61,1154.0,INDIGOPNTS.NS,-8.23,-2.31,0.00,66.0,2673.0,984.0,89.93,132.0,1148.85,20.0,1.0,1.0,H
70,446.0,JUBLFOOD.NS,-12.63,-2.35,0.00,55.0,903.0,422.0,95.01,102.0,439.80,68.0,1.0,1.0,H
66,618.0,IRCTC.NS,-6.39,-1.06,-0.50,64.0,1164.0,375.0,69.20,88.0,609.75,42.0,1.0,1.0,H
33,3512.0,DMART.NS,-8.16,-1.67,0.06,56.0,5324.0,3045.0,79.51,52.0,3476.95,9.0,1.0,1.0,H
64,1253.0,INFY.NS,-15.75,-0.94,-2.46,32.0,1898.0,1223.0,95.56,51.0,1240.70,24.0,1.0,1.0,H
17,1475.0,BATAINDIA.NS,-8.24,-1.80,-0.28,64.0,2173.0,1387.0,88.80,47.0,1483.40,14.0,1.0,1.0,H
39,4413.0,GILLETTE.NS,-9.03,-1.36,-1.18,53.0,5944.0,4186.0,87.09,35.0,4353.00,7.0,0.0,1.0,M
89,40321.0,PAGEIND.NS,-6.95,-1.41,0.44,63.0,53631.0,29040.0,54.13,33.0,38201.00,1.0,1.0,1.0,H
14,6280.0,BAJFINANCE.NS,-2.80,-1.33,-0.33,69.0,7901.0,5265.0,61.49,26.0,5928.85,5.0,1.0,1.0,H
7,2902.0,ASIANPAINT.NS,-3.16,-1.17,0.43,61.0,3566.0,2576.0,67.07,23.0,2827.88,10.0,1.0,1.0,H


In [13]:
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,EstmtFlag,DuraFlag,Conviction
89,40321.0,PAGEIND.NS,-6.95,-1.41,0.44,63.0,53631.0,29040.0,54.13,33.0,38201.00,1.0,1.0,1.0,H
7,2902.0,ASIANPAINT.NS,-3.16,-1.17,0.43,61.0,3566.0,2576.0,67.07,23.0,2827.88,10.0,1.0,1.0,H
93,2419.0,PIDILITIND.NS,-3.62,-1.16,0.38,59.0,2902.0,1991.0,53.02,20.0,2419.95,8.0,1.0,1.0,H
107,764.0,TATACONSUM.NS,1.44,-0.93,0.28,72.0,875.0,655.0,50.45,15.0,702.36,40.0,1.0,1.0,H
98,2420.0,RELIANCE.NS,-1.16,-0.57,0.21,65.0,2811.0,2029.0,50.00,16.0,2353.60,13.0,1.0,0.0,H
48,1688.0,HDFCBANK.NS,6.47,0.76,0.18,63.0,1696.0,1281.0,1.93,0.0,1665.30,18.0,1.0,0.0,H
33,3512.0,DMART.NS,-8.16,-1.67,0.06,56.0,5324.0,3045.0,79.51,52.0,3476.95,9.0,1.0,1.0,H
35,3301.0,EICHERMOT.NS,-0.27,-0.51,0.00,66.0,3850.0,2241.0,34.12,17.0,3167.25,6.0,1.0,1.0,H
61,1154.0,INDIGOPNTS.NS,-8.23,-2.31,0.00,66.0,2673.0,984.0,89.93,132.0,1148.85,20.0,1.0,1.0,H
70,446.0,JUBLFOOD.NS,-12.63,-2.35,0.00,55.0,903.0,422.0,95.01,102.0,439.80,68.0,1.0,1.0,H


In [14]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares'], axis=1, inplace=True)

In [15]:
# all time high sales and profit TTM

star_stocks = new_stars

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(star_stocks)]
tmp_df = tmp_df[~tmp_df['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
1,1925.0,ADANIENT.NS,-34.35,-3.89,-0.06,58.0,4165.0,1194.0,75.40,116.0,1.0,0.0,M
73,4419.0,LTIM.NS,-3.23,-0.42,-0.89,48.0,7439.0,3735.0,81.53,68.0,1.0,0.0,M
15,229.0,BANDHANBNK.NS,-2.84,-2.12,-0.92,67.0,346.0,187.0,73.58,51.0,1.0,1.0,L
56,1079.0,ICICIGI.NS,-6.16,-0.78,-0.55,49.0,1620.0,1060.0,96.61,50.0,1.0,0.0,M
100,772.0,SBICARD.NS,-2.12,-1.52,0.40,59.0,1133.0,673.0,78.48,47.0,1.0,0.0,M
109,201.0,TATAPOWER.NS,-4.69,-1.42,-0.50,58.0,287.0,103.0,46.74,43.0,1.0,0.0,M
18,610.0,BERGEPAINT.NS,3.45,-0.85,0.52,65.0,858.0,540.0,77.99,41.0,1.0,0.0,M
26,908.0,CIPLA.NS,-12.35,-0.97,-1.42,50.0,1173.0,856.0,83.60,29.0,1.0,0.0,M
111,3219.0,TCS.NS,-0.39,0.19,-0.98,53.0,3886.0,2915.0,68.69,21.0,1.0,0.0,M


In [16]:
# star stocks ordered based on slope
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
18,610.0,BERGEPAINT.NS,3.45,-0.85,0.52,65.0,858.0,540.0,77.99,41.0,1.0,0.0,M
100,772.0,SBICARD.NS,-2.12,-1.52,0.40,59.0,1133.0,673.0,78.48,47.0,1.0,0.0,M
1,1925.0,ADANIENT.NS,-34.35,-3.89,-0.06,58.0,4165.0,1194.0,75.40,116.0,1.0,0.0,M
109,201.0,TATAPOWER.NS,-4.69,-1.42,-0.50,58.0,287.0,103.0,46.74,43.0,1.0,0.0,M
56,1079.0,ICICIGI.NS,-6.16,-0.78,-0.55,49.0,1620.0,1060.0,96.61,50.0,1.0,0.0,M
73,4419.0,LTIM.NS,-3.23,-0.42,-0.89,48.0,7439.0,3735.0,81.53,68.0,1.0,0.0,M
15,229.0,BANDHANBNK.NS,-2.84,-2.12,-0.92,67.0,346.0,187.0,73.58,51.0,1.0,1.0,L
111,3219.0,TCS.NS,-0.39,0.19,-0.98,53.0,3886.0,2915.0,68.69,21.0,1.0,0.0,M
26,908.0,CIPLA.NS,-12.35,-0.97,-1.42,50.0,1173.0,856.0,83.60,29.0,1.0,0.0,M


In [17]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS','SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
0,1763.0,ACC.NS,-18.87,-2.39,-1.14,53.0,2748.0,1614.0,86.86,56.0,0.0,0.0,L
5,396.0,AMBUJACEM.NS,-15.46,-1.49,0.54,61.0,588.0,275.0,61.34,48.0,0.0,0.0,L
92,14005.0,PGHH.NS,0.96,-0.17,0.22,53.0,15778.0,12000.0,46.93,13.0,0.0,0.0,L
84,21761.0,NESTLEIND.NS,12.07,0.67,1.12,82.0,21761.0,15919.0,0.00,0.0,1.0,1.0,H
104,3450.0,SIEMENS.NS,14.72,0.83,0.85,67.0,3450.0,1922.0,0.00,0.0,1.0,1.0,H


In [18]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 3


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
63,155.0,INDUSTOWER.NS,-11.93,-2.27,-1.31,67.0,303.0,137.0,89.16,95.0,1.0,1.0,H
15,229.0,BANDHANBNK.NS,-2.84,-2.12,-0.92,67.0,346.0,187.0,73.58,51.0,1.0,1.0,L
49,530.0,HDFCLIFE.NS,-1.53,-0.56,0.60,56.0,757.0,461.0,76.69,43.0,1.0,1.0,H


In [19]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
63,155.0,INDUSTOWER.NS,-11.93,-2.27,-1.31,67.0,303.0,137.0,89.16,95.0,1.0,1.0,H
49,530.0,HDFCLIFE.NS,-1.53,-0.56,0.60,56.0,757.0,461.0,76.69,43.0,1.0,1.0,H
15,229.0,BANDHANBNK.NS,-2.84,-2.12,-0.92,67.0,346.0,187.0,73.58,51.0,1.0,1.0,L


In [20]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 8


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
59,322.0,INDIANB.NS,19.41,2.96,0.70,65.0,322.0,112.0,0.00,0.0,1.0,1.0,H
84,21761.0,NESTLEIND.NS,12.07,0.67,1.12,82.0,21761.0,15919.0,0.00,0.0,1.0,1.0,H
104,3450.0,SIEMENS.NS,14.72,0.83,0.85,67.0,3450.0,1922.0,0.00,0.0,1.0,1.0,H
16,188.0,BANKBARODA.NS,14.48,1.22,1.19,74.0,193.0,71.0,4.10,3.0,1.0,1.0,H
24,317.0,CANBK.NS,8.72,1.37,0.69,72.0,337.0,139.0,10.10,6.0,1.0,1.0,H
55,918.0,ICICIBANK.NS,3.81,0.11,0.69,68.0,953.0,619.0,10.48,4.0,1.0,1.0,H
114,2642.0,TITAN.NS,3.85,-0.08,0.72,66.0,2770.0,1576.0,10.72,5.0,1.0,1.0,H
105,988.0,SUNPHARMA.NS,-0.14,0.71,-0.31,51.0,1044.0,651.0,14.25,6.0,1.0,1.0,H


In [21]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 3


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
84,21761.0,NESTLEIND.NS,12.07,0.67,1.12,82.0,21761.0,15919.0,0.00,0.0,1.0,1.0,H
104,3450.0,SIEMENS.NS,14.72,0.83,0.85,67.0,3450.0,1922.0,0.00,0.0,1.0,1.0,H
55,918.0,ICICIBANK.NS,3.81,0.11,0.69,68.0,953.0,619.0,10.48,4.0,1.0,1.0,H


In [22]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] == 1) & (tmp_df['DuraFlag'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_150'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_150'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 4


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
114,2642.0,TITAN.NS,3.85,-0.08,0.72,66.0,2770.0,1576.0,10.72,5.0,1.0,1.0,H
49,530.0,HDFCLIFE.NS,-1.53,-0.56,0.60,56.0,757.0,461.0,76.69,43.0,1.0,1.0,H
15,229.0,BANDHANBNK.NS,-2.84,-2.12,-0.92,67.0,346.0,187.0,73.58,51.0,1.0,1.0,L
63,155.0,INDUSTOWER.NS,-11.93,-2.27,-1.31,67.0,303.0,137.0,89.16,95.0,1.0,1.0,H


In [23]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['EstmtFlag'] != 1) | (tmp_df['DuraFlag'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 43


,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
4,1029.0,ADANITRANS.NS,-53.04,-9.08,-2.66,52.0,4105.0,643.0,88.85,299.0,1.0,-1.0,L
90,737.0,PEL.NS,-8.82,-2.35,-1.76,61.0,2866.0,635.0,95.43,289.0,-1.0,0.0,L
40,1359.0,GLAND.NS,-14.16,-4.42,0.47,58.0,4316.0,1154.0,93.52,218.0,1.0,0.0,M
2,951.0,ADANIGREEN.NS,-38.80,-5.98,1.55,57.0,2970.0,462.0,80.50,212.0,-1.0,-1.0,L
54,73.0,IBREALEST.NS,2.54,-2.82,1.72,75.0,191.0,47.0,81.94,162.0,-1.0,-1.0,L
1,1925.0,ADANIENT.NS,-34.35,-3.89,-0.06,58.0,4165.0,1194.0,75.40,116.0,1.0,0.0,M
85,58.0,NETWORK18.NS,-6.37,-3.23,0.00,57.0,114.0,40.0,75.68,97.0,-1.0,-1.0,L
120,1323.0,WHIRLPOOL.NS,-8.69,-2.00,0.30,49.0,2510.0,1254.0,94.51,90.0,1.0,0.0,M
123,197.0,ZEEL.NS,-16.28,-2.13,-1.47,45.0,367.0,164.0,83.74,86.0,-1.0,1.0,L
121,385.0,WIPRO.NS,-1.28,-0.77,-0.53,61.0,713.0,356.0,91.88,85.0,-1.0,0.0,L


In [24]:
stock_id = 'MUTHOOTFIN.NS'
df_prec_dev[df_prec_dev['Stock'] == stock_id]

,Close,Stock,Dev%_150,Slope%_150,Slope%_50,RSI,Max,Min,Discount%,Upside%ATH,EstmtFlag,DuraFlag,Conviction
80,1021.0,MUTHOOTFIN.NS,0.56,0.1,0.21,61.0,1651.0,897.0,83.55,62.0,-1.0,0.0,L
